In [1]:
import os
from dask_gateway import Gateway
gateway = Gateway(
    "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
    proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
)

In [3]:
os.environ["X509_USER_PROXY"] = "/work/users/yun79/x509up_u383571"
os.environ["PYTHONPATH"] = "/work/users/yun79/dmitry/another_fork/copperhead_fork2"+ os.environ["PYTHONPATH"]
# os.environ["PYTHONPATH"] = ":/work/users/yun79/dmitry/recreate_valerie/copperhead_fork"
os.environ['PWD'] = "/work/users/yun79/dmitry/another_fork/copperhead_fork2"
os.environ["WORKDIR"] = "/work/users/yun79/dmitry/another_fork/copperhead_fork2"
os.environ['XRD_REQUESTTIMEOUT'] = "2400"

cluster = gateway.new_cluster(
    conda_env = "/depot/cms/kernels/python3",
    worker_cores = 1,#1
    worker_memory = 9,#4 # 6 #9
    env = dict(os.environ),

)

# cluster.scale(20) 
# cluster.scale(5) 
cluster.adapt(minimum=3, maximum=300)
cluster

/depot/cms/kernels/python3/lib/python3.10/site-packages/dask_gateway/client.py:702: GatewayWarning: Adapt with `maximum=300, minimum=3` workers would exceed resource limit of 89 workers. Using `maximum=89, minimum=3` instead.
  warnings.warn(GatewayWarning(msg["msg"]))


In [8]:
os.environ['PYTHONPATH'] 

'/work/users/yun79/dmitry/copperhead_fork2:/work/users/yun79/dmitry/copperhead_fork2:/depot/cms/purdue-af/lhapdf/lib/python3.10/site-packages:/opt/root/lib/:/opt/root/bin/'

In [21]:
import awkward as ak

output = ak.from_parquet("/depot/cms/users/yun79/hmm/copperheadV1clean/rereco_yun_JecDefaultZptOnJecUncOn_Nov20/stage1_output/2018/vbf_powheg_dipole/b829fb315670efd73aa2844e07ea605c.parquet")

output_test = ak.from_parquet("/depot/cms/users/yun79/hmm/copperheadV1clean/test/stage1_output/2018/vbf_powheg_dipole/8cb74eb195e73f5080d84dd2750d72b9.parquet")



In [22]:
np.all(np.isclose(output_test.wgt_nominal,output.wgt_nominal))

True

In [13]:
wgt = ak.ones_like(output.dimuon_mass)
for field in output.fields:
    if ("wgt_" in field) and (field != "wgt_nominal"):
        
        if ("_down" not in field) and ("_up" not in field):
            print(field)
            wgt = wgt* output[field]

wgt_genwgt
wgt_lumi
wgt_muID
wgt_muIso
wgt_muTrig
wgt_nnlops
wgt_pu_wgt


In [18]:
import numpy as np
print(wgt)
print(output.wgt_nominal)
np.all(np.isclose(wgt,output.wgt_nominal))

[3.74e-05, 9.46e-06, 6.15e-07, 2.71e-05, ... 1.99e-05, 1.97e-05, 2.07e-05, 7.2e-06]
[3.74e-05, 9.46e-06, 6.15e-07, 2.71e-05, ... 1.99e-05, 1.97e-05, 2.07e-05, 7.2e-06]


True

In [20]:
# ! ls /depot/cms/users/yun79/hmm/copperheadV1clean/rereco_yun_JecDefaultZptOnJecUncOn_Nov20/stage1_output/2018/vbf_powheg_dipole
! ls /depot/cms/users/yun79/hmm/copperheadV1clean/test/stage1_output/2018/vbf_powheg_dipole

8cb74eb195e73f5080d84dd2750d72b9.parquet


In [2]:
"""
To shutdown the cluster, run this cell"
Equivalently, you can also do "squeue -u <username>" 
if this is the only slurm job you're doing
"""

for cluster_info in gateway.list_clusters():
    gateway.connect(cluster_info.name).shutdown()

In [14]:
options = gateway.cluster_options()  

In [ ]:
# options.worker_cores
options.fields

In [9]:
! squeue -u yun79

':/depot/cms/purdue-af/lhapdf/lib/python3.10/site-packages'